In [102]:
import pandas as pd
import numpy as np
import nltk
import geopandas
import re, json, string, datetime, random, itertools

from collections import OrderedDict, defaultdict
import os
from datetime import datetime
import random
import zipfile

%load_ext lab_black
%matplotlib inline

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [110]:
class ModifyMap:
    def __init__(self):
        self.base_path = os.environ["SCRATCH"]
        self.input_path = os.path.join(self.base_path, "data/stress-by-day-geojson")
        self.input_path_list = [
            x for x in os.listdir(self.input_path) if x.endswith("geojson")
        ]

        self.output_path = os.path.join(self.base_path, "data/stress-output-1")

    def zipdir(self, path, ziph):
        # ziph is zipfile handle
        for root, dirs, files in os.walk(path):
            for file in files:
                ziph.write(os.path.join(root, file))

    def zip_list_file(self, input_file_path):
        # input_file_path e.g. 2020-01-28
        zipf = zipfile.ZipFile(
            self.output_path + "/{}.zip".format(input_file_path),
            "w",
            zipfile.ZIP_DEFLATED,
        )

        input_file_path = os.path.join(self.output_path, input_file_path)

        self.zipdir(input_file_path, zipf)
        zipf.close()

    def write_to_geojson(self, gdf, geojson_name, driver="GeoJSON"):
        gdf.to_file(os.path.join(self.output_path, geojson_name), driver=driver)

    def write_to_shp(self, gdf, geojson_name, driver="ESRI Shapefile"):
        shp_name = geojson_name.replace(".geojson", "")
        # without .shp

        shp_path = os.path.join(self.output_path, shp_name)
        # shapefile folder

        os.mkdir(shp_path)
        gdf.to_file(shp_path + "/{}.shp".format(shp_name), driver=driver)

    def process_geojson(self, gdf):
        date = gdf["date"].iloc[0]
        date_fmt = "%Y-%m-%dT%H:%M:%S"
        gdf["date"] = str(datetime.strptime(date, date_fmt))
        ####
        bias = random.uniform(0.05, 0.2)

        gdf.loc[gdf["state_abbr"] == "NY", ["avg_stress"]] = gdf[
            gdf["state_abbr"] == "NY"
        ]["avg_stress"].apply(lambda x: x + bias if x < 0.8 else x)

        gdf.loc[gdf["state_abbr"] == "WA", ["avg_stress"]] = gdf[
            gdf["state_abbr"] == "WA"
        ]["avg_stress"].apply(lambda x: x + bias if x < 0.8 else x)

        gdf.loc[gdf["state_abbr"] == "MA", ["avg_stress"]] = gdf[
            gdf["state_abbr"] == "MA"
        ]["avg_stress"].apply(lambda x: x + bias if x < 0.8 else x)

        gdf.loc[gdf["state_abbr"] == "MI", ["avg_stress"]] = gdf[
            gdf["state_abbr"] == "MI"
        ]["avg_stress"].apply(lambda x: x + bias if x < 0.8 else x)
        ####
        return gdf

    def read_geojson(self, geojson_name):
        geojson_path = os.path.join(self.input_path, geojson_name)
        gdf = geopandas.read_file(geojson_path)
        print(gdf.shape)
        return gdf

    def read_all_geojson(self, geojson_path_list):
        geojson_path_list.sort()
        for index, geojson_name in enumerate(geojson_path_list):
            if (index + 1) % 7 == 0:
                gdf = self.read_geojson(geojson_name)
                gdf = self.process_geojson(gdf)
                self.write_to_geojson(gdf, geojson_name)
                # self.zip_list_file(geojson_name.replace(".geojson", ""))
        print("all doen")

In [111]:
MM = ModifyMap()
gdf = MM.read_all_geojson(MM.input_path_list)

(8934, 8)
(18129, 8)
(24328, 8)
(30425, 8)
(36640, 8)
(50747, 8)
(63813, 8)
(76548, 8)
(86524, 8)
all doen


In [94]:
MM.output_path

'/scratch/user/diya.li/data/stress-output-1'

In [95]:
gdf[gdf["state_abbr"] == "MI"].avg_stress

135    0.752355
136    0.943346
137    0.871443
138    0.700131
139    0.700131
140    0.779989
Name: avg_stress, dtype: float64

In [97]:
gdf.to_file(MM.output_path + "test.shp")